Karol Janic

kwiecień 2023

zad4.ipynb - Plan monitoringu terenu z kontenerami zawierającymi cenny ładunek.

In [155]:
using JuMP
using GLPK

using JSON

In [156]:
dane_wejsciowe = JSON.parse(read("data4.json", String))

n = dane_wejsciowe["n"]
m = dane_wejsciowe["m"]
k = dane_wejsciowe["k"]

polozenia = dane_wejsciowe["polozenia"]

Dict{String, Any} with 12 entries:
  "kontener_4"  => Dict{String, Any}("x"=>0, "y"=>1)
  "kontener_3"  => Dict{String, Any}("x"=>4, "y"=>0)
  "kontener_5"  => Dict{String, Any}("x"=>1, "y"=>2)
  "kontener_9"  => Dict{String, Any}("x"=>3, "y"=>3)
  "kontener_7"  => Dict{String, Any}("x"=>1, "y"=>3)
  "kontener_8"  => Dict{String, Any}("x"=>2, "y"=>3)
  "kontener_11" => Dict{String, Any}("x"=>1, "y"=>4)
  "kontener_10" => Dict{String, Any}("x"=>0, "y"=>4)
  "kontener_1"  => Dict{String, Any}("x"=>0, "y"=>0)
  "kontener_6"  => Dict{String, Any}("x"=>3, "y"=>2)
  "kontener_2"  => Dict{String, Any}("x"=>2, "y"=>0)
  "kontener_12" => Dict{String, Any}("x"=>4, "y"=>4)

In [157]:
model = Model(GLPK.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

Definicja zmiennych decyzyjnych(informacja czy na danym polu znajduje się kamera) oraz ograniczenie ich wartości do 0 i 1:

In [158]:
@variable(model, x[range(0, n - 1), range(0, m - 1)], Bin)

2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, 0:4
    Dimension 2, 0:4
And data, a 5×5 Matrix{VariableRef}:
 x[0,0]  x[0,1]  x[0,2]  x[0,3]  x[0,4]
 x[1,0]  x[1,1]  x[1,2]  x[1,3]  x[1,4]
 x[2,0]  x[2,1]  x[2,2]  x[2,3]  x[2,4]
 x[3,0]  x[3,1]  x[3,2]  x[3,3]  x[3,4]
 x[4,0]  x[4,1]  x[4,2]  x[4,3]  x[4,4]

Ograniczenie na niemożność umieszczenia kamery w miejscu, gdzie znajduje się kontener:

In [159]:
@constraint(model, 
    [kontener in keys(polozenia)], x[polozenia[kontener]["x"], polozenia[kontener]["y"]] == 0)

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, ["kontener_4", "kontener_3", "kontener_5", "kontener_9", "kontener_7", "kontener_8", "kontener_11", "kontener_10", "kontener_1", "kontener_6", "kontener_2", "kontener_12"]
And data, a 12-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 x[0,1] = 0.0
 x[4,0] = 0.0
 x[1,2] = 0.0
 x[3,3] = 0.0
 x[1,3] = 0.0
 x[2,3] = 0.0
 x[1,4] = 0.0
 x[0,4] = 0.0
 x[0,0] = 0.0
 x[3,2] = 0.0
 x[2,0] = 0.0
 x[4,4] = 0.0

Ograniczenie aby chociaż jedna kamera znajdowała się w zasięgu k od każdego kontenera:

In [160]:
for kontener in keys(polozenia)
    pos_x = polozenia[kontener]["x"]
    pos_y = polozenia[kontener]["y"]

    @constraint(model, 
        sum(x[pos_x, j] for j in max(pos_y - k, 0) : min(pos_y + k, m - 1)) +
        sum(x[i, pos_y] for i in max(pos_x - k, 0) : min(pos_x + k, n - 1)) >= 1)
end

Funkcja kosztu(niezbędna liczba kamer):

In [161]:
@objective(model, Min, 
    sum(x[idx_x, idx_y] for idx_x in range(0, n - 1), idx_y in range(0, m - 1)))

x[0,0] + x[0,1] + x[0,2] + x[0,3] + x[0,4] + x[1,0] + x[1,1] + x[1,2] + x[1,3] + x[1,4] + x[2,0] + x[2,1] + x[2,2] + x[2,3] + x[2,4] + x[3,0] + x[3,1] + x[3,2] + x[3,3] + x[3,4] + x[4,0] + x[4,1] + x[4,2] + x[4,3] + x[4,4]

In [162]:
optimize!(model)

In [163]:
solution_summary(model)

* Solver : GLPK

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "Solution is optimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Objective value    : 7.00000e+00
  Objective bound    : 7.00000e+00
  Relative gap       : 0.00000e+00

* Work counters
  Solve time (sec)   : 9.48906e-05


In [164]:
JuMP.objective_value(model)

7.0

In [165]:
for index_x in range(0, n - 1)
    for index_y in range(0, m - 1)
        print(value(x[index_y, index_x]), " ") 
    end
    println("")
end

0.0 1.0 0.0 1.0 0.0 
0.0 1.0 0.0 0.0 0.0 
0.0 0.0 1.0 0.0 0.0 
1.0 0.0 0.0 0.0 1.0 
0.0 0.0 1.0 0.0 0.0 
